In [242]:
from utils.data import Data
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
import networkx as nx
from collections import defaultdict
import numpy as np

In [3]:
d = Data("data/matmat/2015-11-20/answers.pd")
d.filter_first()
answers = d.get_dataframe_all()
items = d.get_items_df()
skills = d.get_skills_df()

In [53]:
SKILL = "multiplication1"
lvl = "2"
skill_id = skills[skills["name"] == SKILL].index[0]
its = items[items["skill_lvl_"+lvl] == skill_id]
its = its.join(pd.Series(skills["name"], name='skill_name'), on="skill_lvl_3")
ans = answers[answers["item"].isin(its.index)]
ans = ans.join(its["skill_name"], on="item")
ans = ans[ans.join(pd.Series(ans.groupby("student").apply(len), name="count"), on="student")["count"] > 20]

In [61]:
s = list(its["skill_name"].unique())
if np.NaN in s:
    s.remove(np.NaN)
df = pd.DataFrame(index=s, columns=s)
for skill1 in s:
    for skill2 in s:
        students1 = ans[ans["skill_name"] == skill1].groupby("student")["correct"].mean()
        students2 = ans[ans["skill_name"] == skill2].groupby("student")["correct"].mean()
        total = len(set(students1.index) & set(students2.index))
        correct_on_1 = set(students1[students1 == 1].index)
        incorrect_on_2 = set(students2[students2 == 0].index)
        df[skill1][skill2] = len(correct_on_1 & incorrect_on_2) / (total if total else 1)

In [72]:
df.astype(float)
six = sorted(df.index, key=lambda x: sum(map(int, x.split("x")))) 
dff = pd.DataFrame(df, index=six, columns=six)
# df["2"]["1"]
sns.heatmap(dff.astype(float))
plt.show()

In [74]:
TRESHOLD = 0.2
edges = df > TRESHOLD
g = nx.DiGraph()
g.add_nodes_from(df.index)
for u in df.index:
    for v in df.index:
        if edges[u][v]:
            g.add_edge(u, v)
            
components = list(nx.strongly_connected_components(g))
g2 = nx.condensation(g, components)

nodes = set(g2.nodes())
levels = {node: 0 for node in nodes}
level = 0
while len(nodes):
    level += 1
    new_nodes = set()
    for node in nodes:
        new_nodes |= set(g2.neighbors(node))
    for node in new_nodes:
        levels[node] = level
    nodes = new_nodes

pos = {}
order = defaultdict(lambda: 0)
level_counts = {level: list(levels.values()).count(level) for level in set(levels.values())}
for node in g2.nodes():
    level = levels[node]
    print(level_counts[level], max(level_counts.values()))
    pos[node] = [order[level] / level_counts[level] * max(level_counts.values()), level + 0.2 * (order[level] % 2)]
    order[level] += 1
nx.draw(g2, pos=pos)
nx.draw_networkx_labels(g2, pos=pos, labels=dict(enumerate(components)))
plt.show()

(True, False)

In [276]:
df

,1+1,1+2,1+3,1+4,1+5,1+6,1+7,1+8,1+9,2+2,...,2+8,3+3,3+4,3+5,3+6,3+7,4+4,4+5,4+6,5+5
1+1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1+2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1+3,0.025641,0.0217391,0,0.0204082,0.0212766,0.0208333,0,0.0208333,0,0.025,...,0.0212766,0,0.0204082,0.0208333,0.0204082,0.02,0,0.0208333,0.02,0.0243902
1+4,0.025641,0.0217391,0.0204082,0,0.0208333,0,0.0212766,0,0.0208333,0.0243902,...,0,0.025,0.02,0,0,0.02,0.0217391,0.0208333,0.02,0.025
1+5,0.0263158,0.0227273,0.0212766,0.0208333,0,0.0208333,0.0212766,0.0212766,0.0212766,0,...,0.0212766,0.025,0.0204082,0.0208333,0,0.0204082,0,0.0212766,0.0204082,0.025
1+6,0.0769231,0.0888889,0.0625,0.0625,0.0625,0,0.0816327,0.0612245,0.0612245,0.0243902,...,0.0425532,0.05,0.08,0.0408163,0.06,0.0588235,0.0666667,0.04,0.0784314,0.0487805
1+7,0.0789474,0.0697674,0.0425532,0.0638298,0.0638298,0.0612245,0,0.0408163,0.0204082,0.0731707,...,0.0638298,0.0512821,0.0408163,0.0416667,0.0408163,0.0392157,0.0227273,0.0408163,0.0588235,0.0697674
1+8,0.0512821,0.0444444,0.0416667,0.0208333,0.0638298,0.0408163,0.0408163,0,0.04,0.05,...,0.0425532,0.0526316,0.04,0.0204082,0.02,0.0384615,0.0217391,0.04,0.0576923,0.0714286
1+9,0.027027,0.0227273,0.0208333,0.0208333,0.0212766,0.0204082,0,0,0,0.025,...,0.0208333,0.025641,0,0,0,0,0,0,0.0192308,0.0238095
2+2,0,0.0512821,0.05,0.0487805,0.05,0.0243902,0.0487805,0.05,0.025,0,...,0.05,0.0285714,0.047619,0.0487805,0.047619,0.0238095,0.0263158,0.0243902,0.047619,0.0294118
